## Set working directory for imports

In [1]:
import json
import requests
import os

#Set working directory to src on your machine
os.chdir('/Users/justinniestroy-admin/Documents/Round2/Fairscape_Repos/mds_python/src')


## Setup jinja env with function to autolabl links

In [55]:
from jinja2 import Environment, FileSystemLoader
from pydantic import BaseModel, Field
from typing import List, Optional

from fairscape_mds.mds.models.user import User
from fairscape_mds.mds.models.fairscape_base import *

# Set up the Jinja2 environment and load the template
env = Environment(loader=FileSystemLoader('../'))

import re


FAIRSCAPE_URL = '"https://fairscape.pods.uvarc.io/'
def add_link(value):
    """For values that match ark or look like urls add a hyperlink"""
    url_pattern = r'^(http|https)://[^\s]+'
    if re.match(IdentifierPattern, value):
        return f'<a href={FAIRSCAPE_URL}{value}">{value}</a>'
    elif re.match(url_pattern, value):
        return f'<a href="{value}">{value}</a>'
    return value

env.filters['add_link'] = add_link

def convert_to_rdf(json_data):
    # Sample dataset in JSON format

    # Convert JSON string to Python dictionary
    data = json.loads(json_data)

    # Create an RDF graph
    g = Graph()

    # Define a namespace
    ex = Namespace("https://w3id.org/EVI#")

    # Convert JSON data to RDF triples
    for key, value in data.items():
        subject = URIRef(ex["dataset"])
        predicate = URIRef(ex[key])
        if isinstance(value, str):
            # Simple conversion of string values to literals
            obj = Literal(value, datatype=XSD.string)
        elif isinstance(value, int):
            # Example of converting integer values to literals
            obj = Literal(value, datatype=XSD.integer)
        else:
            # Add more conditions for other data types as needed
            obj = Literal(str(value), datatype=XSD.string)
        g.add((subject, predicate, obj))

    # Serialize the graph to RDF/XML and Turtle formats
    rdf_xml_data = g.serialize(format='application/rdf+xml')
    turtle_data = g.serialize(format='turtle')

    return rdf_xml_data, turtle_data



## User Template

In [36]:
# Create an instance of the User class with some data
user = User(
    guid = "ark:99999/fake-user",
    name = 'fake user',
    email="user@example.com",
    password="password123",
    organizations=["Org1", "Org2"],
    projects=["Project1", "Project2"],
    datasets=['ark:99999/fake-dataset', "Dataset2"],
    rocrates=["ROCrate1", "ROCrate2"],
    software=['ark:99999/fake-software', "Software2"],
    computations=["Computation1", "Computation2"]
)
json_data = json.dumps(user.dict(by_alias=True), default=str, indent=2)
rdf, turtle = convert_to_rdf(json_data)
template = env.get_template('./development notebooks/vizualization/user_template.html')

# Render the template with the user data
rendered_html = template.render(user=user, 
                                json = json_data,
                                rdf_xml = rdf,
                                turtle = turtle, 
                                type = 'User')

# Print or save the rendered HTML
with open('../development notebooks/vizualization/jinja_created/user_example.html', 'w') as f:
    f.write(rendered_html)

## Software Example

In [37]:
from fairscape_mds.mds.models.software import Software
sample_software = Software(
    guid = 'ark:99999/fake-software',
    name = 'Software Example',
    description = 'Fake Software to show template working.',
    owner="John Doe",
    distribution=["https://example.com/software/download", "https://example.com/software/mirror"],
    usedBy=['ark:99999/fake-software','ark:99999/fake-computation'],
    sourceOrganization='ark:99999/fake-org',
    includedInDataCatalog="ark:99999/fake-catalog"
)

json_data = json.dumps(sample_software.dict(by_alias=True), default=str, indent=2)
rdf, turtle = convert_to_rdf(json_data)
template = env.get_template('./development notebooks/vizualization/software_template.html')

# Render the template with the user data
rendered_html = template.render(software=sample_software, 
                                json = json_data,
                                rdf_xml = rdf,
                                turtle = turtle, 
                                type = 'Software')

# Print or save the rendered HTML
with open('../development notebooks/vizualization/jinja_created/software_example.html', 'w') as f:
    f.write(rendered_html)


## Dataset Example

In [54]:
from fairscape_mds.mds.models.dataset import Dataset
sample_dataset = Dataset(
    guid = 'ark:99999/fake-dataset',
    metadataType = 'EVI:Dataset',
    name = 'Fake Input Dataset Example',
    description = 'Fake Input Dataset to show template working.',
    owner="John Doe",
    author='Roy Kniestroy',
    distribution=["https://example.com/data/download", "https://example.com/data/mirror"],
    usedBy=['ark:99999/fake-software','ark:99999/fake-computation'],
    sourceOrganization='ark:99999/fake-org',
    includedInDataCatalog="ark:99999/fake-catalog"
)

json_data = json.dumps(sample_dataset.dict(by_alias=True), default=str, indent=2)


template = env.get_template('./development notebooks/vizualization/dataset_template.html')


# Render the template with the user data
rendered_html = template.render(
                dataset=sample_dataset,
                json = json_data,
                rdf_xml=rdf_xml_data,
                turtle=turtle_data,
                type='Dataset'
            )

# Print or save the rendered HTML
with open('../development notebooks/vizualization/jinja_created/dataset_example.html', 'w') as f:
    f.write(rendered_html)


## ROCrate Example

In [42]:
import json
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, XSD

# Sample dataset in JSON format
json_data = json.dumps(sample_dataset.dict(by_alias=True), default=str, indent=2)

# Convert JSON string to Python dictionary
data = json.loads(json_data)

# Create an RDF graph
g = Graph()

# Define a namespace
ex = Namespace("https://w3id.org/EVI#")

# Convert JSON data to RDF triples
for key, value in data.items():
    subject = URIRef(ex["dataset"])
    predicate = URIRef(ex[key])
    if isinstance(value, str):
        # Simple conversion of string values to literals
        obj = Literal(value, datatype=XSD.string)
    elif isinstance(value, int):
        # Example of converting integer values to literals
        obj = Literal(value, datatype=XSD.integer)
    else:
        # Add more conditions for other data types as needed
        obj = Literal(str(value), datatype=XSD.string)
    g.add((subject, predicate, obj))

# Serialize the graph to RDF/XML and Turtle formats
rdf_xml_data = g.serialize(format='application/rdf+xml')
turtle_data = g.serialize(format='turtle')

print("RDF/XML:\n", rdf_xml_data)
print("\nTurtle:\n", turtle_data)


RDF/XML:
 <?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:ns1="https://w3id.org/EVI#@"
   xmlns:ns2="https://w3id.org/EVI#"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="https://w3id.org/EVI#dataset">
    <ns1:id rdf:datatype="http://www.w3.org/2001/XMLSchema#string">ark:99999/fake-dataset</ns1:id>
    <ns1:type rdf:datatype="http://www.w3.org/2001/XMLSchema#string">EVI:Dataset</ns1:type>
    <ns2:name rdf:datatype="http://www.w3.org/2001/XMLSchema#string">Fake Input Dataset Example</ns2:name>
    <ns1:context rdf:datatype="http://www.w3.org/2001/XMLSchema#string">{'@vocab': 'https://schema.org/', 'evi': 'https://w3id.org/EVI#'}</ns1:context>
    <ns2:url rdf:datatype="http://www.w3.org/2001/XMLSchema#string">None</ns2:url>
    <ns2:description rdf:datatype="http://www.w3.org/2001/XMLSchema#string">Fake Input Dataset to show template working.</ns2:description>
    <ns2:license rdf:datatype="http://www.w3.org/2001/XMLSchema#string"> 